Homework 4: Sentiment Analysis - Task 4
----

Names & Sections
----
Names: __YOUR NAMES HERE__ (Write these in every notebook you submit. For each partner, write down whether you are a 4120 or a 6120 student.)

Task 4: Neural Networks (20 points)
----

Next, we'll train a feedforward neural net to work with this data. You'll train one neural net which takes the same input as your Logistic Regression model - a sparse vector representing documents as bags of words.

In [1]:
import sentiment_utils as sutils
import numpy as np

from keras.models import Sequential
from keras.layers import Dense

# you can experiment with having some Dropout layers if you'd like to
# this is not required
from keras.layers import Dropout

# if you want to use this again
from sklearn.feature_extraction.text import CountVectorizer



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davebudhram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# define constants for the files we are using
TRAIN_FILE = "movie_reviews_train.txt"
DEV_FILE = "movie_reviews_dev.txt"

# load in your data and make sure you understand the format
# Do not print out too much so as to impede readability of your notebook
X_train, y_train = sutils.generate_tuples_from_file(TRAIN_FILE)
X_dev, y_dev = sutils.generate_tuples_from_file(DEV_FILE)

# you may use either your sparse vectors or sklearn's CountVectorizer's sparse vectors
# you will experiment with multinomial and binarized representations later


In [3]:
def convert2DArray(array):
  result = []
  for list in array:
    list_text = ""
    for word in list:
      list_text += (word + " ")
    result.append(list_text)
  return result

X_train_strs = convert2DArray(X_train)
X_dev_strs = convert2DArray(X_dev) 
count_vectorizer = CountVectorizer(binary=False)
count_vectorizer.fit(X_train_strs)
X_train_bow = count_vectorizer.transform(X_train_strs)
X_dev_bow = count_vectorizer.transform(X_dev_strs)

print(type(X_train_bow))
# print(np.array(X_train_bow))
print(len(count_vectorizer.vocabulary_))

<class 'scipy.sparse._csr.csr_matrix'>
22596


In [4]:
# Create a feedforward neural network model
# that takes a sparse BoW representation of the data as input
# and makes a binary classification of positive/negative sentiment as output
# you may use any number of hidden layers >= 1 and any number of units in each hidden layer (we recommend between 50-200)
# you may use any activation function on the hidden layers 
# you should use a sigmoid activation function on the output layer
# you should use binary cross-entropy as your loss function
# sgd is an appropriate optimizer for this task
# you should report accuracy as your metric
# you may add Dropout layers if you'd like to

# create/compile your model in this cell

model = Sequential()

# hidden layer
# you can play around with different activation functions
model.add(Dense(units=4, activation='relu', input_dim=22596))



# put in an output layer
# activation function is our classification function
model.add(Dense(units=1, activation='sigmoid'))

model.summary()
# call compile here

# configure the learning process
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 90388     
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 90393 (353.10 KB)
Trainable params: 90393 (353.10 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


How many trainable parameters does your model have? __YOUR ANSWER HERE__

In [5]:
# train your model
# Felix's computer takes about 2 sec for 3 epochs
# reports an accuracy of 0.78 at that point using the sgd optimizer

# Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})
# indicates you should change a list into a numpy array
model.fit(X_train_bow, np.array(y_train), epochs=3, verbose=1)


Epoch 1/3
50/50 [==============================] - 0s 1ms/step - loss: 0.6653 - accuracy: 0.6037
Epoch 2/3
50/50 [==============================] - 0s 1ms/step - loss: 0.6443 - accuracy: 0.6438
Epoch 3/3
50/50 [==============================] - 0s 1ms/step - loss: 0.5778 - accuracy: 0.7275


In [6]:
# make a prediction on the dev set
# then make a classification decision based on that prediction
# predicting all examples takes < 1 sec on Felix's computer
preds = model.predict(X_dev_bow)
preds = [x[0] for x in preds]
preds = [1 if x > .5 else 0 for x in preds]
print(preds)

7/7 [==============================] - 0s 890us/step
[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]


In [7]:
# use the model.evaluate function to report the loss and accuracy on the dev set
dev_loss, dev_accuracy = model.evaluate(X_dev_bow, np.array(y_dev), verbose=1)
print(dev_accuracy)
print(dev_loss)


7/7 [==============================] - 0s 1ms/step - loss: 0.5941 - accuracy: 0.7100
0.7099999785423279
0.5940659642219543


In [8]:
# create the same graph as with NB and LR, with your neural network model instead!
# make sure to re-create your model each time you train it — you don't want to start with
# an already trained network!

# For a model with one hidden layer of 50 units:
# Takes < 15 sec to run on Felix's computer w/ 3 epochs
# Takes < 30 sec to run on Felix's computer w/ 10 epochs
# Takes < 50 sec to run on Felix's computer w/ 20 epochs
# you need not train your model more than 20 epochs
# you should experiment with different numbers of epochs to see how performance varies
# you need not create an experiment that takes > 10 min to run (please do not do this)



Report the f1 scores for your model with the following settings, using the same number of epochs to train in both cases:
- number of epochs used: __YOUR ANSWER HERE__
- multinomial features: __YOUR ANSWER HERE__ 
- binarized features: __YOUR ANSWER HERE__